In [27]:
from dotenv import load_dotenv
import os
import dspy
load_dotenv()
import ujson

Här använder jag istället en lokal modell som inte har så mycket information:

In [28]:
import dspy
lm = dspy.LM('ollama_chat/llama3.2', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

In [29]:
corpus = []

folder_path = "./company_map"

for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            text = f.read().strip()  # Remove any leading/trailing whitespace
            if text:  # Only add non-empty strings
                corpus.append(str(text))  # Ensure it's a string
    except Exception as e:
        print(f"Error reading file {filename}: {str(e)}")

print(f"Collected {len(corpus)} documents")
print("First document preview:", corpus[0][:500] if corpus else "No documents collected")


Collected 7 documents
First document preview: Additional Information
Certifications: NordicTech Solutions is certified under ISO 27001 for information security, ensuring that customer data is handled with the highest security standards.
Environmental Commitment: The company is committed to sustainability, operating all its cloud-based servers with 100% renewable energy, aligning with Sweden’s ambitious climate goals.
Customers: Over 50 active customers in the Nordic region as of October 2025, including leading logistics companies, e-commerc


In [30]:
topk_docs_to_retrieve = 1

embedder = dspy.Embedder('openai/text-embedding-3-small', dimensions=512)
search = dspy.retrievers.Embeddings(embedder=embedder, corpus=corpus, k=topk_docs_to_retrieve)

In [31]:
class RAG(dspy.Signature):
    context: str = dspy.InputField(desc="Context provided to help answer the question")
    question: str = dspy.InputField(desc="Questions asked by the user")
    response: str = dspy.OutputField(desc="Response to the question")

class Model(dspy.Module):
    def __init__(self):
        super().__init__()
        self.retrive = dspy.Retrieve(k=5)
        self.respond = dspy.ChainOfThought(RAG)

    def forward(self, question: str):
        context = search(question).passages
        return self.respond(context=context, question=question)

In [32]:
rag = Model()

In [ ]:
rag(question="Does NordicTech Solutions have any competitors?")

Prediction(
    reasoning='NordicTech Solutions har flera konkurrenter i branschen, som beskrivs i sin egen marknadsföring. Utöver de lokala aktörerna AIStart Sweden och FlowTech, finns det globala ledare som UiPath och Automation Anywhere.',
    response='NordicTech Solutions har flera konkurrenter, bland annat AIStart Sweden, FlowTech, UiPath och Automation Anywhere.'
)

In [35]:
dspy.inspect_history(n=1)





[2025-05-20T12:39:28.639600]

System message:

Your input fields are:
1. `context` (str): Context provided to help answer the question
2. `question` (str): Questions asked by the user
Your output fields are:
1. `reasoning` (str)
2. `response` (str): Response to the question
All interactions will be structured in the following way, with the appropriate values filled in.

[[ ## context ## ]]
{context}

[[ ## question ## ]]
{question}

[[ ## reasoning ## ]]
{reasoning}

[[ ## response ## ]]
{response}

[[ ## completed ## ]]
In adhering to this structure, your objective is: 
        Given the fields `context`, `question`, produce the fields `response`.


User message:

[[ ## context ## ]]
«««
    Competitors
    NordicTech Solutions operates in a competitive industry but has identified several competitive advantages:
    Global Players:
    UiPath: A global leader in robotic process automation (RPA), but their solutions are often costly and complex for smaller businesses.
    Automatio

In [ ]:
rag(question="Vad har NordicTech Solutions för konkurrenter?")